In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
generos = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'ScienceFiction', 'TVMovie', 'Thriller', 'War', 'Western']

def read_parquet(pasta):
    arquivos_parquet = glob.glob(os.path.join(pasta, '*.parquet'))
    lista_df = [pd.read_parquet(arquivo) for arquivo in arquivos_parquet]
    df = pd.concat(lista_df, ignore_index=True)
    return df

In [3]:
df_parquet = read_parquet('./preprocess/movies')[generos]
label = 'Action'
features = generos.copy()
features.remove(label)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_parquet[features], df_parquet[label], test_size=0.3, random_state=42, stratify=df_parquet[label])

print("Antes do Undersampling: {}".format(len(X_train)))

rus = RandomUnderSampler(random_state=42)
X_train, y_train = rus.fit_resample(X_train, y_train)

print("Depois do Undersampling: {}".format(len(X_train)))

Antes do Undersampling: 26976
Depois do Undersampling: 8392


In [9]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Fazendo previsões
y_pred = model.predict(X_test)
y_pred = np.where(y_pred < 0.5, 0, 1)

# Avaliando o modelo
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.95      0.77      0.85      9764
           1       0.38      0.76      0.51      1798

    accuracy                           0.77     11562
   macro avg       0.66      0.77      0.68     11562
weighted avg       0.86      0.77      0.80     11562



In [10]:
# Calculando o número de acertos e erros
acertos = (y_test == y_pred).sum()
erros = (y_test != y_pred).sum()

# Imprimindo os resultados
print(f"Número de instâncias que a predição acertou: {acertos}")
print(f"Número de instâncias que a predição errou: {erros}")

# Alternativamente, você pode usar a acurácia para verificar
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2f}")

Número de instâncias que a predição acertou: 8926
Número de instâncias que a predição errou: 2636
Acurácia: 0.77


In [118]:
predictions_df = pd.DataFrame({
    'predicted_rating': y_test,
    'Y_user': y_pred
})

# Gerando o ranking dos filmes com base nas previsões
random_sample = predictions_df.sample(n=50, random_state=42)
print(random_sample.head(50))

       predicted_rating  Y_user
6389                  1       1
18063                 0       0
1337                  1       0
21975                 0       0
10316                 0       0
21477                 1       1
22191                 1       0
35021                 0       0
15770                 0       0
35196                 0       1
9829                  1       1
15648                 0       0
32679                 0       0
25779                 1       1
20682                 1       1
24248                 0       0
4928                  1       1
7967                  0       0
23181                 0       0
15661                 0       0
25045                 0       0
25931                 1       1
9753                  1       1
25330                 1       1
23325                 1       0
27236                 0       0
576                   1       1
10738                 0       1
10119                 1       1
20997                 0       1
3277    